In [ ]:
#importing all the necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch import flatten #flattening before fc layer
from sklearn.metrics import confusion_matrix
from torchvision.datasets import MNIST  #importing MNIST dataset
from tqdm import tqdm
from torchvision import transforms #for transforming the training and testing data 
from torch.utils.data import DataLoader #Dataloader loads the data batchwise with shuffling in a hassle free manner
from torch.optim import Adam #Adam for GD
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler #performs normalization
from mnist import MNIST as MNIST1 #a python package which has the mnist loaders
from sklearn.metrics import mean_squared_error as mse #for MSE 
from PIL import Image as im #used to convert the given image array to a b/w image
from skimage import img_as_ubyte #preserving 0-255 range for skimage.transform.resize
import skimage.transform #to resize the image
from torchvision.utils import make_grid #to visualize the kernels the tensors

In [ ]:
# Flags for running various parts of the assignment

download_flag = False
load_model    = False
master_dir = os.getcwd() #this is the directory we're working in
mnist_dir     = 'mnist_A2' #directory to store the MNIST dataset
mnist_dir_PCA = "mnist"
mnist_path = os.path.join(master_dir,mnist_dir_PCA) #change name to path where mnist is downloaded to
model_dir     = 'AE_model' #directory containing the AE model
download_dir  = 'C:\\Users\\ABHISHEK\\Downloads\\EE6132Ass3'


if not os.path.exists(os.path.join(master_dir, mnist_dir)):
        os.mkdir(os.path.join(master_dir, mnist_dir)) #make the directory if it doesn't exist

if not os.path.exists(os.path.join(master_dir, model_dir)):
        os.mkdir(os.path.join(master_dir, model_dir)) #make the directory if it doesn't exist
model_path = os.path.join(master_dir, model_dir)+'/'


In [ ]:
def Visualize_Image(test_images):
    
    data_ind  = [6003,416,6754,1605,5055,7965,517,5551,7070,6420] #indices of various digits from the test set
    
    for i,ind in enumerate(data_ind):
        
        #visualizing the image just for clarity
        image_sample = np.asarray(test_images[ind],dtype = np.uint8).reshape(28,28)

        #creating image object
        image_obj = im.fromarray(image_sample) #converts the array into a b/w image 28x28

        #saving the image
        image_obj.save(download_dir+'\\'+str(i) +'.png')
        print('Digit '+str(i)+' image has been saved')
        image_obj.show()
        

def Visualize_Reconstr_PCA(reconstructed_data):
    
    data_ind  = [6003,416,6754,1605,5055,7965,517,5551,7070,6420] #indices of various digits from the test set
    
    for i,ind in enumerate(data_ind):
        
        #visualizing the image just for clarity
        image_sample = np.asarray(255*reconstructed_data[ind],dtype = np.uint8).reshape(28,28)

        #creating image object
        image_obj = im.fromarray(image_sample) #converts the array into a b/w image 28x28

        #saving the image
        image_obj.save(download_dir+'\\'+str(i) +' PCA reconstructed.png')
        print('Digit '+str(i)+' image has been saved')
        image_obj.show()
        
    
    
        
        
    

In [ ]:
#defining a class with the required AE architecture

class AE(nn.Module):
    
    def __init__(self): #class constructor
        super(AE,self).__init__() #calls the parent constructor
        
        #initializing the encoder module
        self.encoder = nn.Sequential(nn.Linear(784,512),nn.ReLU(),nn.Linear(512,256),nn.ReLU(),nn.Linear(256,128),nn.ReLU(),nn.Linear(128,30))
        
        #initializing the decoder module
        self.decoder = nn.Sequential(nn.Linear(30,128),nn.ReLU(),nn.Linear(128,256),nn.ReLU(),nn.Linear(256,784),nn.ReLU())
        
    def forward(self,x): #defines the forward pass and also the structure of the network thus helping backprop
        
        x                   = flatten(x,1) #flatten the image to a 784x1 vector
        encoded_input       = self.encoder(x.float())
        reconstructed_input = self.decoder(encoded_input)
        
        return reconstructed_input,encoded_input
    

class AE_1h(nn.Module):
    
    def __init__(self,hidden_layer): #class constructor
        super(AE_1h,self).__init__() #calls the parent constructor
        
        #initializing the encoder module
        self.encoder = nn.Sequential(nn.Linear(784,hidden_layer),nn.ReLU())
        
        #initializing the decoder module
        self.decoder = nn.Sequential(nn.Linear(hidden_layer,784),nn.ReLU())
        
    def forward(self,x): #defines the forward pass and also the structure of the network thus helping backprop
        
        x                   = flatten(x,1) #flatten the image to a 784x1 vector
        encoded_input       = self.encoder(x.float())
        reconstructed_input = self.decoder(encoded_input)
        
        return reconstructed_input,encoded_input
    

class AE_manifold(nn.Module):
    
    def __init__(self): #class constructor
        super(AE_manifold,self).__init__() #calls the parent constructor
        
        #initializing the encoder module
        self.encoder = nn.Sequential(nn.Linear(784,64),nn.ReLU(),nn.Linear(64,8),nn.ReLU())
        
        #initializing the decoder module
        self.decoder = nn.Sequential(nn.Linear(8,64),nn.ReLU(),nn.Linear(64,784),nn.ReLU())
        
    def forward(self,x): #defines the forward pass and also the structure of the network thus helping backprop
        
        x                   = flatten(x,1) #flatten the image to a 784x1 vector
        encoded_input       = self.encoder(x.float())
        reconstructed_input = self.decoder(encoded_input)
        
        return reconstructed_input,encoded_input
        
class conv_AE_unpool(nn.Module): #define unpooling outside the decoder and separately in forward nn.Sequential just takes one input
    
    def __init__(self): #class constructor
        super(conv_AE_unpool,self).__init__() #calls the parent constructor
        
        #initializing the encoder module
        self.encoder_conv1 = nn.Sequential(nn.Conv2d(1,8, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2),return_indices = True)) # 28x28x1 to 14x14x8
        self.encoder_conv2 = nn.Sequential(nn.Conv2d(8,16, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2),return_indices = True)) #14x14x8 to 7x7x16
        self.encoder_conv3 = nn.Sequential(nn.Conv2d(16,16, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2),return_indices = True)) #7x7x16 to 3x3x16
        
        #initializing the decoder module
        self.decoder_conv1 = nn.Sequential(nn.Identity()) #7x7x16 to 7x7x16
        self.decoder_conv2 = nn.Sequential(nn.Conv2d(16,8, kernel_size = 3, stride = 1,padding= 1),nn.ReLU()) #14x14x16 to 14x14x8
        self.decoder_conv3 = nn.Sequential(nn.Conv2d(8,1, kernel_size = 3, stride = 1,padding= 1),nn.ReLU()) #28x28x8 to 28x28x1
        
        #defining the unpooling operation
        self.unpool = nn.MaxUnpool2d(kernel_size = (2,2))
        
        
    def forward(self,x): #defines the forward pass and also the structure of the network thus helping backprop
        
        encoded_input,indices1  = self.encoder_conv1(x.float())  # 28x28x1 to 14x14x8
        encoded_input,indices2  = self.encoder_conv2(encoded_input) #14x14x8 to 7x7x16
        encoded_input,indices3  = self.encoder_conv3(encoded_input) #7x7x16 to 3x3x16
        
        
        reconstructed_input     = self.unpool(encoded_input,indices3,output_size=torch.Size([batch_size, 16, 7, 7])) #3x3x16 to 7x7x16
        reconstructed_input     = self.decoder_conv1(reconstructed_input) #7x7x16 to 7x7x16
        reconstructed_input     = self.unpool(reconstructed_input,indices2) #7x7x16 to 14x14x16
        reconstructed_input     = self.decoder_conv2(reconstructed_input)#14x14x16 to 14x14x8
        reconstructed_input     = self.unpool(reconstructed_input,indices1)#14x14x8 to 28x28x8
        reconstructed_input     = self.decoder_conv3(reconstructed_input)#28x28x8 to 28x28x1

        
        return reconstructed_input,encoded_input
    
class conv_AE_deconv(nn.Module):
    def __init__(self): #class constructor
        super(conv_AE_deconv,self).__init__() #calls the parent constructor
        
        #initializing the encoder module
        self.encoder_conv1 = nn.Sequential(nn.Conv2d(1,8, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2)))
        self.encoder_conv2 = nn.Sequential(nn.Conv2d(8,16, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2)))
        self.encoder_conv3 = nn.Sequential(nn.Conv2d(16,16, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2)))
        
        #initializing the decoder module
        self.decoder_conv1 = nn.Sequential(nn.ConvTranspose2d(16,16, kernel_size = 3, stride = 2),nn.ReLU())
        self.decoder_conv2 = nn.Sequential(nn.ConvTranspose2d(16,8, kernel_size = 4, stride = 2, padding = 1),nn.ReLU())
        self.decoder_conv3 = nn.Sequential(nn.ConvTranspose2d(8,1, kernel_size = 4, stride = 2, padding = 1),nn.ReLU())
        
    def forward(self,x): #defines the forward pass and also the structure of the network thus helping backprop
        
        encoded_input  = self.encoder_conv1(x.float())
        encoded_input  = self.encoder_conv2(encoded_input)
        encoded_input  = self.encoder_conv3(encoded_input)

        reconstructed_input     = self.decoder_conv1(encoded_input)
        reconstructed_input     = self.decoder_conv2(reconstructed_input)
        reconstructed_input     = self.decoder_conv3(reconstructed_input)

        return reconstructed_input,encoded_input

class conv_AE_deconv_unpool(nn.Module):
    def __init__(self): #class constructor
        super(conv_AE_deconv_unpool,self).__init__() #calls the parent constructor
        
         #initializing the encoder module
        self.encoder_conv1 = nn.Sequential(nn.Conv2d(1,8, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2),return_indices = True))
        self.encoder_conv2 = nn.Sequential(nn.Conv2d(8,16, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2),return_indices = True))
        self.encoder_conv3 = nn.Sequential(nn.Conv2d(16,16, kernel_size = 3, stride = 1,padding= 1),nn.ReLU(),nn.MaxPool2d(kernel_size = (2,2),return_indices = True))
        
        #initializing the decoder module
        self.decoder_conv1 = nn.Sequential(nn.ConvTranspose2d(16,16, kernel_size = 3, stride = 1, padding = 1),nn.ReLU())
        self.decoder_conv2 = nn.Sequential(nn.ConvTranspose2d(16,8, kernel_size = 3, stride = 1, padding = 1),nn.ReLU())
        self.decoder_conv3 = nn.Sequential(nn.ConvTranspose2d(8,1, kernel_size = 3, stride = 1, padding = 1),nn.ReLU())
        
        #defining the unpooling operation
        self.unpool = nn.MaxUnpool2d(kernel_size = (2,2))
        
    def forward(self,x): #defines the forward pass and also the structure of the network thus helping backprop
        
        encoded_input,indices1  = self.encoder_conv1(x.float())
        encoded_input,indices2  = self.encoder_conv2(encoded_input)
        encoded_input,indices3  = self.encoder_conv3(encoded_input)
        
        
        reconstructed_input     = self.unpool(encoded_input,indices3,output_size=torch.Size([batch_size, 16, 7, 7]))
        reconstructed_input     = self.decoder_conv1(reconstructed_input)
        reconstructed_input     = self.unpool(reconstructed_input,indices2)
        reconstructed_input     = self.decoder_conv2(reconstructed_input)
        reconstructed_input     = self.unpool(reconstructed_input,indices1)
        reconstructed_input     = self.decoder_conv3(reconstructed_input)
        
        
        return reconstructed_input,encoded_input        
        
    
    

In [ ]:
#Defining a function to train the network: Returns the training loss for the current epoch 
def Train(model,device,TrainDataLoader,optimizer,lossfn,train_length,sparse = False,l1_reg = 0.001,denoise = False,noise_val=0.1,model_flag = 0):  
    
    model.train() #setting the model in training mode
    
    #initializing the total training loss to 0
    train_loss    = 0
     
    #loop over the training set
    
    for (data,label) in tqdm(TrainDataLoader):  # (data,label): Training data for that batch
        
        (data,label) = (data.to(device),label.to(device))  #sending the data to the device we've chosen
        
        if((denoise == False)and (model_flag < 3)):
            #perform forward pass and compute the loss
        
            reconstruction,encoded = model(data) #our reconstruction
            loss = lossfn(reconstruction,flatten(data,1)) #loss 
        
            if(sparse == True):
                loss += l1_reg*torch.linalg.norm(encoded,1) #imposing L1 penalty on the hidden layer activation for sparse AE
            
            optimizer.zero_grad() #zeroing out the gradients before backprop
            loss.backward()       #backprop from the loss
            optimizer.step()      #updating the weights
        
        elif(denoise == True):
            noisy_data = Add_Noise(data, noise_val)
            
            #perform forward pass and compute the loss
        
            reconstruction,encoded = model(noisy_data) #our reconstruction
            loss = lossfn(reconstruction,flatten(data,1)) #loss wrt original data
            
            optimizer.zero_grad() #zeroing out the gradients before backprop
            loss.backward()       #backprop from the loss
            optimizer.step()      #updating the weights
        
        elif(model_flag >= 3):
            
            #perform forward pass and compute the loss
            reconstruction,encoded = model(data) #our reconstruction
            loss = lossfn(reconstruction,data) #loss
            
            optimizer.zero_grad() #zeroing out the gradients before backprop
            loss.backward()       #backprop from the loss
            optimizer.step()      #updating the weights
            
            
        
        #Adding this loss to  training loss and computing correct predictions
        
        train_loss    += loss/train_length
    
    return train_loss #returning loss


#Defining a function to test the network: Returns the test loss for the current epoch
def Test(model,device,TestDataLoader,lossfn,test_length,sparse = False,l1_reg = 0.001,denoise = False,noise_val=0.1,model_flag = 0):
    
    model.eval()  #setting the model in eval/test mode
    
    #initializing the total test loss and total correct test predictions to 0
    test_loss    = 0
    
    #switching off the gradient for eval
    with torch.no_grad():
        
        #loop over the test set
        
        for (data,label) in TestDataLoader: # (data,label): Test data for that batch
            
            (data,label) = (data.to(device),label.to(device))  #sending the data to the device we've chosen
        
            
            if((denoise == False)and (model_flag < 3)):
                #perform forward pass and compute the loss
                reconstruction,encoded = model(data) #our prediction
                loss = lossfn(reconstruction,flatten(data,1)) #loss 
            
                if(sparse == True):
                    
                    loss += l1_reg*torch.linalg.norm(encoded,1) #imposing L1 penalty on the hidden layer activation for sparse AE
            
            elif(denoise == True):
                noisy_data = Add_Noise(data, noise_val)
            
                #perform forward pass and compute the loss
        
                reconstruction,encoded = model(noisy_data) #our reconstruction
                loss = lossfn(reconstruction,flatten(data,1)) #loss wrt original data
            
            elif(model_flag >= 3):
            
                #perform forward pass and compute the loss
        
                reconstruction,encoded = model(data) #our reconstruction
                loss = lossfn(reconstruction,data) #loss
            
                
            #Adding this loss to  test loss
        
            test_loss    += loss/test_length
    
    return test_loss #returning loss
def average_act(model,device,TestDataLoader,lossfn,test_length):
    model.eval()  #setting the model in eval/test mode
    
    #initializing the total test loss and total correct test predictions to 0
    average_act_val    = 0
    
    #switching off the gradient for eval
    with torch.no_grad():
        
        #loop over the test set
        
        for (data,label) in TestDataLoader: # (data,label): Test data for that batch
            
            (data,label) = (data.to(device),label.to(device))  #sending the data to the device we've chosen
        
            
        
            #perform forward pass and compute the average activation
            reconstruction,encoded = model(data) #our prediction 
                    
            average_act_val += float(torch.mean(encoded))        
                
        
        
    average_act_val    /= test_length
    
    print('The average activation norm is ',average_act_val)
            
        
    
def Test_Image(model,model_name,device,img,name):
    
    img = torch.from_numpy(img)
    
    print('Loading Model')
        
    model.load_state_dict(torch.load(model_path+model_name), strict=False)
    
    with torch.no_grad():
        if(device == torch.device("cuda")): #if we're working on a GPU
            test_image = img.reshape(1,1,28,28).cuda().float() #reshaping the image into 28x28 pixels

        else:
            test_image = img.reshape(1,1,28,28).float()
                    
            
        #detach breaks the image from the computational graph layer of the tensor before converting it to numpy format

        reconstructed_image,encoded = model.forward(test_image) #as it is a single image we directly run the forward pass
                
        plt.imshow(reconstructed_image.reshape(28,28),cmap = plt.cm.gray, interpolation='nearest',clim=(0, 255)) #our reconstructed image
        str_title = "Reconstructed image: "+name
        plt.title(str_title)
        plt.savefig(download_dir+'\\'+'Reconstructed_Image_'+name+model_name+'.png')
        plt.show()
    
def Visualize_activations(model,TestDataLoader,model_name,device,hidden_layer): #visualize the activations
    data_ind  = [6003,416,6754,1605,5055,7965,517,5551,7070,6420] #indices of various digits from the test set
    
    print('Loading Model')
        
    model.load_state_dict(torch.load(model_path+model_name), strict=False)
    
    for i,ind in enumerate(data_ind):
            
        test_image = TestDataLoader.dataset.data[ind].clone()  #creates a copy of the test_image from the dataset
            
        with torch.no_grad():
            if(device == torch.device("cuda")): #if we're working on a GPU
                test_image = test_image.reshape(1,1,28,28).cuda().float() #reshaping the image into 28x28 pixels

            else:
                test_image = test_image.reshape(1,1,28,28).float()
                    
            
            #detach breaks the image from the computational graph layer of the tensor before converting it to numpy format

            reconstructed_image,encoded = model.forward(test_image) #as it is a single image we directly run the forward pass
                    
            encoded = encoded.detach().cpu().numpy()
            plt.imshow(encoded.reshape(int(np.sqrt(hidden_layer)),int(np.sqrt(hidden_layer)))) #our activation image assuming hidden layer size is 900
            str_title = "Activation for digit "+str(i)
            plt.title(str_title)
            plt.savefig(download_dir+'\\'+'Activation_Image_'+str(i)+model_name+'.png')
            plt.show()

def Visualize_Filters(model,model_name,device): #visualize the weights for ten neurons spaced
    
    print('Loading Model')
        
    model.load_state_dict(torch.load(model_path+model_name), strict=False)
    
    with torch.no_grad():
        
        encoder_filters = model.encoder[0].weight.detach().cpu().numpy()
        decoder_filters = model.decoder[0].weight.detach().cpu().numpy()
        
        #plot the encoder and decoder weights as an image
        
        for i in range(10):
        
            plt.imshow(encoder_filters[10*i].reshape(28,28))
            plt.colorbar()
            plt.title('Encoder Filters for '+str(10*i)+'th neuron')
            plt.savefig(download_dir+'\\'+'Encoder_filters'+str(10*i)+model_name+'.png')
            plt.show()
        
            plt.imshow(decoder_filters[:,10*i].reshape(28,28))
            plt.colorbar()
            plt.title('Decoder Filters for '+str(10*i)+'th neuron')
            plt.savefig(download_dir+'\\'+'Decoder_filters'+str(10*i)+model_name+'.png')
            plt.show()
    
def Add_Noise(image, noise_val = 0.3 ): #adding salt and pepper noise 

    noise = torch.randn(image.size())*noise_val
    noisy_image = image + noise
    return noisy_image

def Manifold_Analysis(model,TestDataLoader,model_name,device,noise_val):
    data_ind  = [6003,416,6754,1605,5055,7965,517,5551,7070,6420] #indices of various digits from the test set
    
    print('Loading Model')
        
    model.load_state_dict(torch.load(model_path+model_name), strict=False)
    
    for i,ind in enumerate(data_ind):
            
        test_image = TestDataLoader.dataset.data[ind].clone()  #creates a copy of the test_image from the dataset
            
        with torch.no_grad():
            if(device == torch.device("cuda")): #if we're working on a GPU
                test_image = test_image.reshape(1,1,28,28).cuda().float() #reshaping the image into 28x28 pixels

            else:
                test_image = test_image.reshape(1,1,28,28).float()
                    
            
            encoded = model.encoder[3].forward(test_image) #obtaining the encoded output
            encoded = Add_Noise(encoded,noise_val) #adding noise to the feature before passing it onto the decoder
            reconstructed_image = model.decoder[3].forward(encoded)
            
            reconstructed_image = reconstructed_image.detach().cpu().numpy()
            plt.imshow(reconstructed_image.reshape(28,28),cmap = plt.cm.gray, interpolation='nearest',clim=(0, 255)) #our reconstructed image
            str_title = "Reconstructed image: "+str(i)
            plt.title(str_title)
            plt.savefig(download_dir+'\\'+'Reconstructed_Image_Manifold'+str(i)+str(noise_val)+model_name+'.png')
            plt.show()
            
            
            
def Visualize_Decoder_Weights(model,model_name,device,model_flag):
    
    model.load_state_dict(torch.load(model_path+model_name), strict=False) #loading the model
    
    if(model_flag == 3): #CAE with just unpooling
            
        conv_2_filter = model.decoder_conv2[0].weight.detach().clone() #creating a copy of the filter weights
        
        if(device == torch.device('cuda')):
            conv_2_filter = conv_2_filter.cpu() #get it to the CPU
            
        #normalizing the filters by scaling the values this makes it stand out from the background
        conv_2_filter -= conv_2_filter.min()
        conv_2_filter /= conv_2_filter.max()
        
        #all the filters are block filters
        #Therefore, using a random number generator to choose which block filters to plot
        filt_ind = np.random.randint(0 ,conv_2_filter.size()[0],3)
        
        for ind in filt_ind:  
            print(conv_2_filter[ind].size())

            image         = make_grid(conv_2_filter[ind].reshape(16,1,3,3)) #this returns a tensor grid containing the images
            print(image.size())
            image         = image.permute(1,2,0) #permuting the dimensions of the tensor grid to make it right 

            #plotting the second layer filters
            plt.imshow(image)
            str_title = 'Decoder Second Convolutional Layer Filter no: ' + str(ind)
            plt.title(str_title)
            plt.savefig(download_dir+'\\'+'DecoderConv2'+str(ind)+model_name+'.png')
            plt.show()
        
        conv_3_filter = model.decoder_conv3[0].weight.detach().clone() #creating a copy of the filter weights
        
        if(device == torch.device('cuda')):
            conv_3_filter = conv_3_filter.cpu() #get it to the CPU
            
        #normalizing the filters by scaling the values this makes it stand out from the background
        conv_3_filter -= conv_3_filter.min()
        conv_3_filter /= conv_3_filter.max()
        print(conv_3_filter.size())
        image         = make_grid(conv_3_filter.reshape(8,1,3,3)) #this returns a tensor grid containing the images
        image         = image.permute(1,2,0) #permuting the dimensions of the tensor grid to make it right 

        #plotting the second layer filters
        plt.imshow(image)
        str_title = 'Decoder Third Convolutional Layer Filter'
        plt.title(str_title)
        plt.savefig(download_dir+'\\'+'DecoderConv3'+str(ind)+model_name+'.png')
        plt.show()    
        
    
    elif(model_flag == 4): #CAE with just deconvolution
        print(model)
        conv_1_filter = model.decoder_conv1[0].weight.detach().clone() #creating a copy of the filter weights
        
        if(device == torch.device('cuda')):
            conv_1_filter = conv_1_filter.cpu() #get it to the CPU
            
        #normalizing the filters by scaling the values this makes it stand out from the background
        conv_1_filter -= conv_1_filter.min()
        conv_1_filter /= conv_1_filter.max()
        
        #all the filters are block filters
        #Therefore, using a random number generator to choose which block filters to plot
        filt_ind = np.random.randint(0 ,conv_1_filter.size()[0],3)
        
        for ind in filt_ind:    

            image         = make_grid(conv_1_filter[ind].reshape(16,1,3,3)) #this returns a tensor grid containing the images
            image         = image.permute(1,2,0) #permuting the dimensions of the tensor grid to make it right 

            #plotting the second layer filters
            plt.imshow(image)
            str_title = 'Decoder First Convolutional Layer Filter no: ' + str(ind)
            plt.title(str_title)
            plt.savefig(download_dir+'\\'+'DecoderConv1'+str(ind)+model_name+'.png')
            plt.show()
            
        conv_2_filter = model.decoder_conv2[0].weight.detach().clone() #creating a copy of the filter weights
        
        if(device == torch.device('cuda')):
            conv_2_filter = conv_2_filter.cpu() #get it to the CPU
            
        #normalizing the filters by scaling the values this makes it stand out from the background
        conv_2_filter -= conv_2_filter.min()
        conv_2_filter /= conv_2_filter.max()
        
        #all the filters are block filters
        #Therefore, using a random number generator to choose which block filters to plot
        filt_ind = np.random.randint(0 ,conv_2_filter.size()[0],3)
        
        for ind in filt_ind:   
            print(conv_2_filter[ind].size())

            image         = make_grid(conv_2_filter[ind].reshape(8,1,4,4)) #this returns a tensor grid containing the images
            image         = image.permute(1,2,0) #permuting the dimensions of the tensor grid to make it right 

            #plotting the second layer filters
            plt.imshow(image)
            str_title = 'Decoder Second Convolutional Layer Filter no: ' + str(ind)
            plt.title(str_title)
            plt.savefig(download_dir+'\\'+'DecoderConv2'+str(ind)+model_name+'.png')
            plt.show()
        
        conv_3_filter = model.decoder_conv3[0].weight.detach().clone() #creating a copy of the filter weights
        
        if(device == torch.device('cuda')):
            conv_3_filter = conv_3_filter.cpu() #get it to the CPU
            
        #normalizing the filters by scaling the values this makes it stand out from the background
        conv_3_filter -= conv_3_filter.min()
        conv_3_filter /= conv_3_filter.max()
        print(conv_3_filter.size())
        
        image         = make_grid(conv_3_filter.reshape(8,1,4,4)) #this returns a tensor grid containing the images
        image         = image.permute(1,2,0) #permuting the dimensions of the tensor grid to make it right 

        #plotting the second layer filters
        plt.imshow(image)
        str_title = 'Decoder Third Convolutional Layer Filter'
        plt.title(str_title)
        plt.savefig(download_dir+'\\'+'DecoderConv3'+str(ind)+model_name+'.png')
        plt.show()
        
    
    elif(model_flag == 5): #CAE with both deconvolution and unpooling
        conv_1_filter = model.decoder_conv1[0].weight.detach().clone() #creating a copy of the filter weights
        
        if(device == torch.device('cuda')):
            conv_1_filter = conv_1_filter.cpu() #get it to the CPU
            
        #normalizing the filters by scaling the values this makes it stand out from the background
        conv_1_filter -= conv_1_filter.min()
        conv_1_filter /= conv_1_filter.max()
        
        #all the filters are block filters
        #Therefore, using a random number generator to choose which block filters to plot
        filt_ind = np.random.randint(0 ,conv_1_filter.size()[0],3)
        
        for ind in filt_ind:    

            image         = make_grid(conv_1_filter[ind].reshape(16,1,3,3)) #this returns a tensor grid containing the images
            image         = image.permute(1,2,0) #permuting the dimensions of the tensor grid to make it right 

            #plotting the second layer filters
            plt.imshow(image)
            str_title = 'Decoder First Convolutional Layer Filter no: ' + str(ind)
            plt.title(str_title)
            plt.savefig(download_dir+'\\'+'DecoderConv1:'+str(ind)+model_name+'.png')
            plt.show()
            
        conv_2_filter = model.decoder_conv2[0].weight.detach().clone() #creating a copy of the filter weights
        
        if(device == torch.device('cuda')):
            conv_2_filter = conv_2_filter.cpu() #get it to the CPU
            
        #normalizing the filters by scaling the values this makes it stand out from the background
        conv_2_filter -= conv_2_filter.min()
        conv_2_filter /= conv_2_filter.max()
        
        #all the filters are block filters
        #Therefore, using a random number generator to choose which block filters to plot
        filt_ind = np.random.randint(0 ,conv_2_filter.size()[0],3)
        
        for ind in filt_ind: 

            image         = make_grid(conv_2_filter[ind].reshape(8,1,3,3)) #this returns a tensor grid containing the images
            image         = image.permute(1,2,0) #permuting the dimensions of the tensor grid to make it right 

            #plotting the second layer filters
            plt.imshow(image)
            str_title = 'Decoder Second Convolutional Layer Filter no: ' + str(ind)
            plt.title(str_title)
            plt.savefig(download_dir+'\\'+'DecoderConv2:'+str(ind)+model_name+'.png')
            plt.show()
        
        conv_3_filter = model.decoder_conv3[0].weight.detach().clone() #creating a copy of the filter weights
        
        if(device == torch.device('cuda')):
            conv_3_filter = conv_3_filter.cpu() #get it to the CPU
            
        #normalizing the filters by scaling the values this makes it stand out from the background
        conv_3_filter -= conv_3_filter.min()
        conv_3_filter /= conv_3_filter.max()
        
        image         = make_grid(conv_3_filter.reshape(8,1,3,3)) #this returns a tensor grid containing the images
        image         = image.permute(1,2,0) #permuting the dimensions of the tensor grid to make it right 

        #plotting the second layer filters
        plt.imshow(image)
        str_title = 'Decoder Third Convolutional Layer Filter '
        plt.title(str_title)
        plt.savefig(download_dir+'\\'+'DecoderConv3:'+str(ind)+model_name+'.png')
        plt.show()
        


def Run_AE(model_name,load_model,model_flag,hidden_layer=256,sparse = False,l1_reg = 0.001,pltmode = 0,img = 0,name = '0',denoise = False,noise_val = 0.1):
    
    app_transform = transforms.ToTensor() #convert the images to tensor datatype
    
    #organize the training and test data
    
    train_data    = MNIST(mnist_dir, train = True, download = download_flag, transform = app_transform) #getting training data
    test_data     = MNIST(mnist_dir, train = False, transform = app_transform)
    
    #initialize the dataloaders
    TrainDataLoader = DataLoader(train_data, batch_size = batch_size, shuffle = True ) 
    TestDataLoader  = DataLoader(test_data, batch_size = batch_size) 
    
    train_length  = len(TrainDataLoader.dataset) #no of training examples
    test_length   = len(TestDataLoader.dataset)  #no of testing cases
    
    #model
    if(model_flag == 0):
        model = AE().to(device)
    elif(model_flag == 1):
        model = AE_1h(hidden_layer=hidden_layer).to(device)
    
    elif(model_flag == 2):
        model = AE_manifold().to(device)
        
    elif(model_flag == 3):
        model = conv_AE_unpool().to(device)
        #model = CNN().to(device)
        
    elif(model_flag == 4):
        model = conv_AE_deconv().to(device)
    
    elif(model_flag == 5):
        model = conv_AE_deconv_unpool().to(device)
    
    #initialize the optimizer
    optimizer = Adam(model.parameters(),lr = learning_rate) #using Adam for GD as its the fastest and state of the art
    
    #loss function:MSE
    lossfn = nn.MSELoss()
    
    if(load_model): #load a pre-trained model
        if(pltmode == 0):
            print('Loading Model')
        
            model.load_state_dict(torch.load(model_path+model_name), strict=False)
            
        
            #viewing the reconstructed images
            data_ind  = [6003,416,6754,1605,5055,7965,517,5551,7070,6420] #indices of various digits from the test set
        
            for i,ind in enumerate(data_ind):
            
                test_image = TestDataLoader.dataset.data[ind].clone()  #creates a copy of the test_image from the dataset
                
                if(denoise == True):
                    noisy_image = Add_Noise(test_image,noise_val)
                    test_image  = noisy_image
                    noisy_image = noisy_image.detach().cpu().numpy()
                    
                    
                    
                    #plot the noisy image
                    plt.imshow(noisy_image.reshape(28,28),cmap='gray') #our noisy image
                    str_title = "Noisy image "+str(i)
                    plt.title(str_title)
                    plt.savefig(download_dir+'\\'+'Noisy_Image_'+str(i)+model_name+'.png')
                    plt.show()
                    
                    
            
                with torch.no_grad():
                    if(device == torch.device("cuda")): #if we're working on a GPU
                        test_image = test_image.reshape(1,1,28,28).cuda().float() #reshaping the image into 28x28 pixels

                    else:
                        test_image = test_image.reshape(1,1,28,28).float()
                    
            
                    #detach breaks the image from the computational graph layer of the tensor before converting it to numpy format

                    reconstructed_image,encoded = model.forward(test_image) #as it is a single image we directly run the forward pass
                    
                    reconstructed_image = reconstructed_image.detach().cpu().numpy()
                    plt.imshow(reconstructed_image.reshape(28,28),cmap='gray') #our reconstructed image
                    str_title = "Reconstructed image "+str(i)
                    plt.title(str_title)
                    plt.savefig(download_dir+'\\'+'Reconstructed_Image_'+str(i)+model_name+'.png')
                    plt.show()
            
    
        if(pltmode == 1):
            Test_Image(model,model_name,device,img,name)
            
        if(pltmode == 2):
            Visualize_activations(model,TestDataLoader,model_name,device,hidden_layer)
            
        if(pltmode == 3):
            Visualize_Filters(model,model_name,device)
            
        if(pltmode == 4):
            Manifold_Analysis(model,TestDataLoader,model_name,device,noise_val)
        
        if(pltmode == 5):
            Visualize_Decoder_Weights(model,model_name,device,model_flag)
            
        if(pltmode == 6):
            average_act(model,device,TestDataLoader,lossfn,test_length)
            
            
                
        
    else:
        
        #initialising the lists
        
        train_losses   = []
        test_losses    = []
        
        for epoch in range(1, N_epochs+1):
            print("Epoch ",epoch," has just begun!")
            print('****************** ', epoch/N_epochs," % ******************") #creates a status bar instead of using tqdm
            
            #train the model
            loss = Train(model,device,TrainDataLoader,optimizer,lossfn,train_length,sparse = sparse,l1_reg = l1_reg,denoise = denoise,noise_val=noise_val,model_flag=model_flag)
            train_losses.append(loss)
            print('Train loss for Epoch ',epoch,': ',loss)
    
            #test the model
            loss = Test(model,device,TestDataLoader,lossfn,test_length,sparse = sparse,l1_reg = l1_reg,denoise = denoise,noise_val=noise_val,model_flag=model_flag)
            test_losses.append(loss)
            print('Test loss for Epoch ',epoch,': ',loss)
            
            
        #plotting the loss curves
        
        figure,axes = plt.subplots(2,1,constrained_layout = True,sharex = True)
        
        axes[0].plot(np.asfarray(train_losses),'o-',label = 'train losses')
        axes[0].set_ylabel('Loss')
        axes[0].grid()
        axes[0].legend()
        
        axes[1].plot(np.asfarray(test_losses),'o-',label= 'test losses')
        axes[1].set_ylabel('Loss')
        axes[1].set_xlabel('Iterations')
        axes[1].grid()
        axes[1].legend()
        
        figure.set_figwidth(8)
        figure.set_figheight(10)
        figure.suptitle("Training and testing plots for the Autoencoder",x = 0.5,y=1.1)
        figure.savefig(download_dir+'\\'+model_name+'_loss.png')
        figure.show()
        
        
        
        #showing final test loss
        
        final_loss = Test(model,device,TestDataLoader,lossfn,test_length,sparse = sparse,l1_reg = l1_reg,denoise = denoise,noise_val=noise_val,model_flag=model_flag)
        
        print('Final Reconstruction Loss = ',float(final_loss))
        
        # Save the model we just trained
        torch.save(model.state_dict(), model_path+model_name)
        print(model)
        
    
    
    

In [ ]:
#Training Hyperparams:
learning_rate = 1e-3
batch_size    = 64
N_epochs      = 10 


#device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #checks for gpu else runs in cpu



In [ ]:
def Run_Assignment():

    answer = str(input('Do you want to check the results of question 1? (y/n)'))
    
    if(answer == 'y'):
        
        #Data Pre-processing
        print('Pre-processing data......')
        #getting the dataset ready
        #Reusing code from assignment 1

        #creating an mnist object

        mnist_obj = MNIST1(mnist_path)
        mnist_obj.gz = True #as I downloaded the .gz zip files
        train_images,train_labels = mnist_obj.load_training() #loads the training dataset
        test_images,test_labels   = mnist_obj.load_testing() #loads the testing dataset

        print("Done extracting the data")    

        #modifications to training_data and testing_data

        train_data = np.asarray(train_images)/255
        test_data = np.asarray(test_images)/255
        
        answer = str(input('Do you want to take a look at the model test images?'))
        
        if(answer == 'y'):
            Visualize_Image(test_images)
            
        
        #running the PCA part of the assignment
        print('Running the PCA with 30 principal components')
        
        PCA_data = np.concatenate((train_data,test_data))
        
        pca1 = PCA(n_components = 30) #as we take first 30 eigenvalues
        pca1.fit(PCA_data)
        train_pca = pca1.transform(PCA_data)
        reconstructed_data = pca1.inverse_transform(train_pca)
        PCA_error = mse(PCA_data,reconstructed_data)
        print('Reconstruction error made by PCA: ',PCA_error)
        
        answer = str(input('Do you want to visualize the reconstructed images through PCA?'))
        
        if(answer == 'y'):
            Visualize_Reconstr_PCA(reconstructed_data)
            
            
        #running the AE part of this question
        
        print('Running the AE part....')
        
        model_name = 'Vanilla_AE.mdl'
        
        load_model = False
        
        model_flag = 0
        
        #Run_AE(model_name,load_model,model_flag)
        
        
        answer = str(input('Do you want to visualize the reconstructed images through AE?'))
        
        if(answer == 'y'):
            
            model_name = 'Vanilla_AE.mdl'
    
            model_flag = 0
            
            load_model = True
            
            Run_AE(model_name,load_model,model_flag)
            
    answer = str(input('Do you want to check the results of question 2? (y/n)'))
    
    if(answer == 'y'):
        
        
        x = [64,128,256] #size of the hidden layer
        
        for hidden_layer in x:

            
            model_name = 'AE_h_'+str(hidden_layer)+'.mdl'
        
            load_model = False
        
            model_flag = 1

            #Run_AE(model_name,load_model,model_flag,hidden_layer=hidden_layer)
            
            answer = str(input('Do you want to visualize the reconstructed images through AE?'))
        
            if(answer == 'y'):
            
                load_model = True
            
                Run_AE(model_name,load_model,model_flag,hidden_layer=hidden_layer) 
                
        
        #testing the output of a non-digit image on the autoencoder
        
        answer = str(input('Do you want to check the output of the standard autoencoder when a non digit image is passed to it?'))
        
        if(answer == 'y'):
            lena = im.open('lena.png').convert('L') #converts rgb image to gray scale
            lena.save(download_dir+'\\'+'lena_greyscale.png') #saving the gray scale image
            lena = np.asarray(lena)
            x = img_as_ubyte(skimage.transform.resize(lena, (28,28))) #resizing to 28x28 pixels
        
            plt.imshow(x,cmap = plt.cm.gray, interpolation='nearest',clim=(0, 255))
            plt.savefig(download_dir+'\\'+'lena_28x28.png')
            plt.show()
            
            load_model = True
            Run_AE(model_name,load_model,model_flag,hidden_layer=hidden_layer,pltmode = 1,img = x,name = 'lena')
                
    
    answer = str(input('Do you want to check the results of question 3? (y/n)'))
    
    if(answer == 'y'):
        
        #train an overcomplete sparse Autoencoder
        
        x = 900 #hidden layer size 
            
        reg_vals = [0.001,0.1,1] #try three different values of regularization
                
        for reg in reg_vals:
            
            model_name = 'AE_h_sparse'+str(reg)+str(x)+'.mdl'
        
            load_model = False
        
            model_flag = 1

            #Run_AE(model_name,load_model,model_flag,hidden_layer=x,sparse = True,l1_reg = reg)
            
            answer = str(input('Do you want to visualize the reconstructed images through AE?'))
        
            if(answer == 'y'):
            
                load_model = True
                Run_AE(model_name,load_model,model_flag,hidden_layer=x,sparse = True,l1_reg = reg) #produce reconstructed images
                    
                Run_AE(model_name,load_model,model_flag,hidden_layer=x,pltmode = 2,sparse = True,l1_reg = reg) #produce activation images
                    
                Run_AE(model_name,load_model,model_flag,hidden_layer=x,pltmode = 3,sparse = True,l1_reg = reg) #produce filter images
                    
        answer = str(input('Do you want to visualize the filters and the activations for the standard AE?'))
            
        if(answer == 'y'):
                
            model_name = 'AE_h_'+str(256)+'.mdl' #largest standard AE
        
            load_model = True
        
            model_flag = 1
                
            Run_AE(model_name,load_model,model_flag,hidden_layer=256,pltmode = 2) #produce activation images
                    
            Run_AE(model_name,load_model,model_flag,hidden_layer=256,pltmode = 3) #produce filter images
                
                    
    answer = str(input('Do you want to check the results of question 4? (y/n)'))
    
    if(answer == 'y'): 
        
        #train a denoising autoencoder with 256 hidden layer neurons
        x = 256
        
        noise_vals = [0.1,0.3,0.7] #three different values of noise that we can use
        
        for noise_val in noise_vals:
        
            model_name = 'AE_h_denoise'+str(noise_val)+str(x)+'.mdl'
        
            load_model = False
        
            model_flag = 1

            #Run_AE(model_name,load_model,model_flag,hidden_layer=x,denoise = True,noise_val = noise_val)
            
            answer = str(input('Do you want to visualize the reconstructed images through AE?'))
        
            if(answer == 'y'):
            
                load_model = True
                Run_AE(model_name,load_model,model_flag,hidden_layer=x,denoise = True,noise_val = noise_val)
                #Run_AE(model_name,load_model,model_flag,hidden_layer=x,pltmode = 3,denoise = True,noise_val = noise_val) #produce filter images
        
            
        #testing noisy image on standard autoencoder
        answer = str(input('Do you want to visualize the reconstructed images for a noisy input through AE?'))
        if(answer == 'y'):
            noisy_digit = im.open('Noisy_Image_8AE_h_denoise0.7256.mdl.png').convert('L') #converts rgb image to gray scale
            noisy_digit = np.asarray(noisy_digit)
            x_img = img_as_ubyte(skimage.transform.resize(noisy_digit, (28,28))) #resizing to 28x28 pixels
        
            plt.imshow(x_img,cmap = plt.cm.gray, interpolation='nearest',clim=(0, 255))
            plt.savefig(download_dir+'\\'+'noisy_image_28x28.png')
            plt.show()
            
            model_name = 'AE_h_'+str(x)+'.mdl'
        
            model_flag = 1
            
            load_model = True
            Run_AE(model_name,load_model,model_flag,hidden_layer=x,pltmode = 1,img = x_img,name = 'noisy_image')
                
        
    answer = str(input('Do you want to check the results of question 5? (y/n)'))
    
    if(answer == 'y'): 
        
        
        #training the AE for the manifold analysis
        
        model_name = 'AE_manifold.mdl'
        
        load_model = False
        
        model_flag = 2

        #Run_AE(model_name,load_model,model_flag)
        
        answer = str(input('Do you want to visualize the reconstructed images?'))
        if(answer == 'y'):
            
            load_model = True
            
            noise_vals = [0.1,0.3,0.7] #three different values of noise that we can use
            
            for noise_val in noise_vals:
            
                Run_AE(model_name,load_model,model_flag,pltmode = 4,noise_val = noise_val)
            
    
    answer = str(input('Do you want to check the results of question 6? (y/n)'))
    
    if(answer == 'y'):
        
        answer = str(input('Do you want to check the results of question conv AE with unpooling? (y/n)'))
    
        if(answer == 'y'):
            
            #training the conv AE with unpooling
            
            model_name = 'conv_AE_unpool.mdl'
        
            load_model = False
        
            model_flag = 3
            
            Run_AE(model_name,load_model,model_flag)
            
        answer = str(input('Do you want to check the results of question conv AE with deconvolution? (y/n)'))
    
        if(answer == 'y'):
            
            #training the conv AE with deconvolution
            
            model_name = 'conv_AE_deconv.mdl'
        
            load_model = False
        
            model_flag = 4
            
            Run_AE(model_name,load_model,model_flag)
            
        answer = str(input('Do you want to check the results of question conv AE with unpooling and deconvolution? (y/n)'))
    
        if(answer == 'y'):
            
            #training the conv AE with deconvolution and unpooling
            
            model_name = 'conv_AE_unpool_deconv.mdl'
        
            load_model = False
        
            model_flag = 5
            
            Run_AE(model_name,load_model,model_flag)

        
        
            
        
        
        
            
        
        
            
        
        
        
        
        
        
        
            
        
        
        
        
        
        
        
    

In [ ]:
model_name = 'AE_h_'+str(256)+'.mdl'
        
load_model = True
        
model_flag = 1

Run_AE(model_name,load_model,model_flag,hidden_layer=256,pltmode = 6)

x = 900 #hidden layer size 
            
reg_vals = [0.001,0.1,1] #try three different values of regularization
                
for reg in reg_vals:
            
    model_name = 'AE_h_sparse'+str(reg)+str(x)+'.mdl'
        
    load_model = True
        
    model_flag = 1

    Run_AE(model_name,load_model,model_flag,hidden_layer=x,sparse = True,l1_reg = reg,pltmode = 6)
            
